In [ ]:
# !git clone https://github.com/Sridipta-Roy/EEG-Classification.git

Cloning into 'EEG-Classification'...
remote: Enumerating objects: 594, done.
remote: Total 594 (delta 0), reused 0 (delta 0), pack-reused 594 (from 2)
Receiving objects: 100% (594/594), 141.83 MiB | 15.70 MiB/s, done.
Resolving deltas: 100% (8/8), done.
Updating files: 100% (666/666), done.


In [ ]:
# %pip install -r /content/EEG-Classification/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 136.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 120.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 98.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 152.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 96.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.8/59.8 kB 8.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
from tqdm import tqdm
import joblib

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (classification_report, confusion_matrix,
                            accuracy_score, precision_score, recall_score,
                            f1_score, roc_auc_score, roc_curve)

import tensorflow as tf
from tensorflow import keras

# Custom modules
import sys
sys.path.append('../src')
#sys.path.append('/content/EEG-Classification/src')
from models import TraditionalMLModels, DeepLearningModels, get_callbacks

warnings.filterwarnings('ignore')

In [2]:
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Set random seeds
np.random.seed(42)
tf.random.set_seed(42)

In [3]:
print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

TensorFlow version: 2.19.0
GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
DATA_DIR = Path('../data/processed/bonn')
FEATURES_DIR = DATA_DIR / 'features'
MODELS_DIR = Path('../results/models')
FIGURES_DIR = Path('../results/figures')

# DATA_DIR = Path('/content/EEG-Classification/data/processed/bonn')
# FEATURES_DIR = DATA_DIR / 'features'
# MODELS_DIR = Path('/content/EEG-Classification/results/models')
# FIGURES_DIR = Path('/content/EEG-Classification/results/figures')

# Create directories
MODELS_DIR.mkdir(parents=True, exist_ok=True)
FIGURES_DIR.mkdir(parents=True, exist_ok=True)

In [5]:
# Training configuration
TRAINING_CONFIG = {
    'test_size': 0.15,
    'val_size': 0.15,
    'random_state': 42,
    'epochs': 50,
    'batch_size': 32,
    'patience': 10,
    'learning_rate': 0.001,
}

In [7]:
# ============================================================================
# LOAD DATA
# ============================================================================

In [6]:
print("\n" + "="*80)
print("LOADING DATA")
print("="*80)

# Load feature-based data for traditional ML
print("\n1. Loading engineered features for traditional ML...")
X_train_features = np.load(FEATURES_DIR / 'X_train_features.npy')
y_train = np.load(FEATURES_DIR / 'y_train.npy')
X_val_features = np.load(FEATURES_DIR / 'X_val_features.npy')
y_val = np.load(FEATURES_DIR / 'y_val.npy')
X_test_features = np.load(FEATURES_DIR / 'X_test_features.npy')
y_test = np.load(FEATURES_DIR / 'y_test.npy')

print(f" Training features: {X_train_features.shape}")
print(f" Validation features: {X_val_features.shape}")
print(f" Test features: {X_test_features.shape}")


LOADING DATA

1. Loading engineered features for traditional ML...
 Training features: (5600, 74)
 Validation features: (1200, 74)
 Test features: (1200, 74)


In [7]:
# Load raw signals for deep learning
print("\n2. Loading raw preprocessed signals for deep learning...")
splits_dir = DATA_DIR / 'splits'
X_train_raw = np.load(splits_dir / 'X_train.npy')
X_val_raw = np.load(splits_dir / 'X_val.npy')
X_test_raw = np.load(splits_dir / 'X_test.npy')

print(f" Training signals: {X_train_raw.shape}")
print(f" Validation signals: {X_val_raw.shape}")
print(f" Test signals: {X_test_raw.shape}")



2. Loading raw preprocessed signals for deep learning...
 Training signals: (5600, 694)
 Validation signals: (1200, 694)
 Test signals: (1200, 694)


In [12]:
# ============================================================================
#   DATA PREPARATION
# ============================================================================

In [8]:
# Standardize features for traditional ML
print("\nStandardizing features for traditional ML...")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_features)
X_val_scaled = scaler.transform(X_val_features)
X_test_scaled = scaler.transform(X_test_features)

print(f"Features standardized (mean≈0, std≈1)")
print(f"Training: mean={X_train_scaled.mean():.6f}, std={X_train_scaled.std():.6f}")


Standardizing features for traditional ML...
Features standardized (mean≈0, std≈1)
Training: mean=0.000000, std=1.000000


In [14]:
# Save scaler for future use
joblib.dump(scaler, MODELS_DIR / 'feature_scaler.pkl')
print(f"Saved scaler to {MODELS_DIR}/feature_scaler.pkl")


Saved scaler to /content/EEG-Classification/results/models/feature_scaler.pkl


In [9]:
# Reshape raw signals for deep learning
print("\nReshaping signals for deep learning...")
# Add channel dimension: (samples, timesteps) -> (samples, timesteps, channels)
X_train_dl = X_train_raw.reshape(X_train_raw.shape[0], X_train_raw.shape[1], 1)
X_val_dl = X_val_raw.reshape(X_val_raw.shape[0], X_val_raw.shape[1], 1)
X_test_dl = X_test_raw.reshape(X_test_raw.shape[0], X_test_raw.shape[1], 1)

print(f"Reshaped for DL: {X_train_dl.shape}")
print(f"Input shape: (timesteps={X_train_dl.shape[1]}, channels={X_train_dl.shape[2]})")



Reshaping signals for deep learning...
Reshaped for DL: (5600, 694, 1)
Input shape: (timesteps=694, channels=1)


In [16]:
# ============================================================================
# TRADITIONAL ML MODELS
# ============================================================================

In [17]:
#Dictionary to store results
ml_results = {}

def train_and_evaluate_ml(model, model_name, X_train, y_train, X_val, y_val, X_test, y_test):
    """Train and evaluate a traditional ML model"""
    print(f"\n{'='*60}")
    print(f"Training {model_name}")
    print(f"{'='*60}")

    # Train
    print(f"  Training...")
    model.fit(X_train, y_train)

    # Predictions
    y_train_pred = model.predict(X_train)
    y_val_pred = model.predict(X_val)
    y_test_pred = model.predict(X_test)

    # Probabilities (for ROC-AUC)
    if hasattr(model, 'predict_proba'):
        y_train_proba = model.predict_proba(X_train)[:, 1]
        y_val_proba = model.predict_proba(X_val)[:, 1]
        y_test_proba = model.predict_proba(X_test)[:, 1]
    else:
        y_train_proba = y_train_pred
        y_val_proba = y_val_pred
        y_test_proba = y_test_pred

    # Metrics
    results = {
        'model': model,
        'train_acc': accuracy_score(y_train, y_train_pred),
        'train_precision': precision_score(y_train, y_train_pred),
        'train_recall': recall_score(y_train, y_train_pred),
        'train_f1': f1_score(y_train, y_train_pred),
        'train_auc': roc_auc_score(y_train, y_train_proba),
        'val_acc': accuracy_score(y_val, y_val_pred),
        'val_precision': precision_score(y_val, y_val_pred),
        'val_recall': recall_score(y_val, y_val_pred),
        'val_f1': f1_score(y_val, y_val_pred),
        'val_auc': roc_auc_score(y_val, y_val_proba),
        'test_acc': accuracy_score(y_test, y_test_pred),
        'test_precision': precision_score(y_test, y_test_pred),
        'test_recall': recall_score(y_test, y_test_pred),
        'test_f1': f1_score(y_test, y_test_pred),
        'test_auc': roc_auc_score(y_test, y_test_proba),
        'y_test_pred': y_test_pred,
        'y_test_proba': y_test_proba
    }

    # Print results
    print(f"\n  Results:")
    print(f"    Training   - Acc: {results['train_acc']:.4f}, F1: {results['train_f1']:.4f}, AUC: {results['train_auc']:.4f}")
    print(f"    Validation - Acc: {results['val_acc']:.4f}, F1: {results['val_f1']:.4f}, AUC: {results['val_auc']:.4f}")
    print(f"    Test       - Acc: {results['test_acc']:.4f}, F1: {results['test_f1']:.4f}, AUC: {results['test_auc']:.4f}")

    # Save model
    joblib.dump(model, MODELS_DIR / f'{model_name.lower().replace(" ", "_")}.pkl')
    print(f"Saved model")

    return results


In [18]:
# 1. Logistic Regression (Baseline)
print("\n1. LOGISTIC REGRESSION (Baseline)")
lr = TraditionalMLModels.get_logistic_regression()
ml_results['Logistic Regression'] = train_and_evaluate_ml(
    lr, 'Logistic Regression',
    X_train_scaled, y_train, X_val_scaled, y_val, X_test_scaled, y_test
)


1. LOGISTIC REGRESSION (Baseline)

Training Logistic Regression
  Training...

  Results:
    Training   - Acc: 0.9855, F1: 0.9855, AUC: 0.9993
    Validation - Acc: 0.9833, F1: 0.9833, AUC: 0.9990
    Test       - Acc: 0.9817, F1: 0.9816, AUC: 0.9991
Saved model


In [19]:
# 2. Random Forest
print("\n2. RANDOM FOREST")
rf = TraditionalMLModels.get_random_forest(n_estimators=100)
ml_results['Random Forest'] = train_and_evaluate_ml(
    rf, 'Random Forest',
    X_train_scaled, y_train, X_val_scaled, y_val, X_test_scaled, y_test
)



2. RANDOM FOREST

Training Random Forest
  Training...

  Results:
    Training   - Acc: 1.0000, F1: 1.0000, AUC: 1.0000
    Validation - Acc: 0.9925, F1: 0.9925, AUC: 0.9999
    Test       - Acc: 0.9900, F1: 0.9900, AUC: 0.9998
Saved model


In [20]:
# 3. XGBoost
print("\n3. XGBOOST")
xgb = TraditionalMLModels.get_xgboost(n_estimators=100)
ml_results['XGBoost'] = train_and_evaluate_ml(
    xgb, 'XGBoost',
    X_train_scaled, y_train, X_val_scaled, y_val, X_test_scaled, y_test
)


3. XGBOOST

Training XGBoost
  Training...

  Results:
    Training   - Acc: 1.0000, F1: 1.0000, AUC: 1.0000
    Validation - Acc: 0.9958, F1: 0.9958, AUC: 0.9997
    Test       - Acc: 0.9942, F1: 0.9942, AUC: 0.9998
Saved model


In [21]:
# 4. LightGBM
print("\n4. LIGHTGBM")
lgb = TraditionalMLModels.get_lightgbm(n_estimators=100)
ml_results['LightGBM'] = train_and_evaluate_ml(
    lgb, 'LightGBM',
    X_train_scaled, y_train, X_val_scaled, y_val, X_test_scaled, y_test
)


4. LIGHTGBM

Training LightGBM
  Training...

  Results:
    Training   - Acc: 1.0000, F1: 1.0000, AUC: 1.0000
    Validation - Acc: 0.9967, F1: 0.9967, AUC: 0.9998
    Test       - Acc: 0.9958, F1: 0.9958, AUC: 0.9999
Saved model


In [22]:
# 5. SVM
print("\n5. SUPPORT VECTOR MACHINE")
svm = TraditionalMLModels.get_svm(kernel='rbf')
ml_results['SVM'] = train_and_evaluate_ml(
    svm, 'SVM',
    X_train_scaled, y_train, X_val_scaled, y_val, X_test_scaled, y_test
)


5. SUPPORT VECTOR MACHINE

Training SVM
  Training...

  Results:
    Training   - Acc: 0.9955, F1: 0.9955, AUC: 0.9999
    Validation - Acc: 0.9942, F1: 0.9942, AUC: 0.9999
    Test       - Acc: 0.9958, F1: 0.9958, AUC: 0.9998
Saved model


In [23]:
# ============================================================================
# DEEP LEARNING MODELS
# ============================================================================


In [ ]:
# Dictionary to store results
dl_results = {}

def train_and_evaluate_dl(model, model_name, X_train, y_train, X_val, y_val, X_test, y_test, epochs=100):
    """Train and evaluate a deep learning model"""
    print(f"\n{'='*60}")
    print(f"Training {model_name}")
    print(f"{'='*60}")

    # Model summary
    print(f"\n  Model Architecture:")
    model.summary()
    print(f"  Total parameters: {model.count_params():,}")

    # Callbacks
    model_callbacks = get_callbacks(model_name.lower().replace(" ", "_"))

    # Train
    print(f"\n  Training...")
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=32,
        callbacks=model_callbacks,
        verbose=0
    )

    # Predictions
    y_train_proba = model.predict(X_train, verbose=0).flatten()
    y_val_proba = model.predict(X_val, verbose=0).flatten()
    y_test_proba = model.predict(X_test, verbose=0).flatten()

    y_train_pred = (y_train_proba > 0.5).astype(int)
    y_val_pred = (y_val_proba > 0.5).astype(int)
    y_test_pred = (y_test_proba > 0.5).astype(int)

    # Metrics
    results = {
        'model': model,
        'history': history,
        'train_acc': accuracy_score(y_train, y_train_pred),
        'train_precision': precision_score(y_train, y_train_pred),
        'train_recall': recall_score(y_train, y_train_pred),
        'train_f1': f1_score(y_train, y_train_pred),
        'train_auc': roc_auc_score(y_train, y_train_proba),
        'val_acc': accuracy_score(y_val, y_val_pred),
        'val_precision': precision_score(y_val, y_val_pred),
        'val_recall': recall_score(y_val, y_val_pred),
        'val_f1': f1_score(y_val, y_val_pred),
        'val_auc': roc_auc_score(y_val, y_val_proba),
        'test_acc': accuracy_score(y_test, y_test_pred),
        'test_precision': precision_score(y_test, y_test_pred),
        'test_recall': recall_score(y_test, y_test_pred),
        'test_f1': f1_score(y_test, y_test_pred),
        'test_auc': roc_auc_score(y_test, y_test_proba),
        'y_test_pred': y_test_pred,
        'y_test_proba': y_test_proba,
        'epochs_trained': len(history.history['loss'])
    }

    # Print results
    print(f"\n  Results (after {results['epochs_trained']} epochs):")
    print(f"    Training   - Acc: {results['train_acc']:.4f}, F1: {results['train_f1']:.4f}, AUC: {results['train_auc']:.4f}")
    print(f"    Validation - Acc: {results['val_acc']:.4f}, F1: {results['val_f1']:.4f}, AUC: {results['val_auc']:.4f}")
    print(f"    Test       - Acc: {results['test_acc']:.4f}, F1: {results['test_f1']:.4f}, AUC: {results['test_auc']:.4f}")

    # Save model
    #model.save(MODELS_DIR / f'{model_name.lower().replace(" ", "_")}.h5')
    model.save(MODELS_DIR / f'{model_name.lower().replace(" ", "_")}.keras')
    print(f"Saved model")

    return results

In [11]:
# Input shape
input_shape = (X_train_dl.shape[1], X_train_dl.shape[2])
print(f"\nInput shape for DL models: {input_shape}")


Input shape for DL models: (694, 1)


In [30]:
# 6. 1D CNN
print("\n6. 1D CONVOLUTIONAL NEURAL NETWORK")
cnn_model = DeepLearningModels.build_1d_cnn(input_shape)
dl_results['1D CNN'] = train_and_evaluate_dl(
    cnn_model, '1D CNN',
    X_train_dl, y_train, X_val_dl, y_val, X_test_dl, y_test,
    epochs=TRAINING_CONFIG['epochs']
)


6. 1D CONVOLUTIONAL NEURAL NETWORK

Training 1D CNN

  Model Architecture:


Model: "1D_CNN"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 694, 1)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 694, 64)        │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn_1 (BatchNormalization)       │ (None, 694, 64)        │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ relu_1 (Activation)             │ (None, 694, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ maxpool_1 (MaxPooling1D)        │ (None, 347, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 347, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 347, 128)       │        41,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn_2 (BatchNormalization)       │ (None, 347, 128)       │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ relu_2 (Activation)             │ (None, 347, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ maxpool_2 (MaxPooling1D)        │ (None, 173, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 173, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 173, 256)       │        98,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn_3 (BatchNormalization)       │ (None, 173, 256)       │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ relu_3 (Activation)             │ (None, 173, 256)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ maxpool_3 (MaxPooling1D)        │ (None, 86, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 86, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_avg_pool                 │ (None, 256)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_dense (Dropout)         │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 183,169 (715.50 KB)

 Trainable params: 182,273 (712.00 KB)

...


Epoch 31: val_loss improved from 0.00693 to 0.00623, saving model to ../results/models/1d_cnn_best.h5



Epoch 32: val_loss did not improve from 0.00623

Epoch 33: val_loss did not improve from 0.00623

Epoch 34: val_loss improved from 0.00623 to 0.00576, saving model to ../results/models/1d_cnn_best.h5



Epoch 35: val_loss did not improve from 0.00576

Epoch 36: val_loss did not improve from 0.00576

Epoch 37: val_loss did not improve from 0.00576

Epoch 38: val_loss did not improve from 0.00576

Epoch 39: val_loss improved from 0.00576 to 0.00459, saving model to ../results/models/1d_cnn_best.h5



Epoch 40: val_loss did not improve from 0.00459

Epoch 41: val_loss did not improve from 0.00459

Epoch 42: val_loss did not improve from 0.00459

Epoch 43: val_loss did not improve from 0.00459

Epoch 44: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.

Epoch 44: val_loss did not improve from 0.00459

Epoch 45: val_loss did not improve from 0.00459

Epoch 46: val_loss did not improve from 0.00459

Epoch 47: val_loss did not improve from 0.00459

Epoch 48: val_loss did not improve from 0.00459

Epoch 49: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.

Epoch 49: val_loss did not improve from 0.00459
Epoch 49: early stopping
Restoring model weights from the end of the best epoch: 39.



  Results (after 49 epochs):
    Training   - Acc: 0.9973, F1: 0.9973, AUC: 1.0000
    Validation - Acc: 0.9975, F1: 0.9975, AUC: 1.0000
    Test       - Acc: 0.9967, F1: 0.9967, AUC: 1.0000
Saved model


In [12]:
# 7. LSTM
print("\n7. LSTM (Bidirectional)")
lstm_model = DeepLearningModels.build_lstm(input_shape, bidirectional=True)
dl_results['LSTM'] = train_and_evaluate_dl(
    lstm_model, 'LSTM',
    X_train_dl, y_train, X_val_dl, y_val, X_test_dl, y_test,
    epochs=TRAINING_CONFIG['epochs']
)


7. LSTM (Bidirectional)

Training LSTM

  Model Architecture:


Model: "LSTM"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 694, 1)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bilstm_1 (Bidirectional)        │ (None, 694, 256)       │       133,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bilstm_2 (Bidirectional)        │ (None, 128)            │       164,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_dense (Dropout)         │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 305,793 (1.17 MB)

 Trainable params: 305,793 (1.17 MB)

 Non-trainable params: 0 (0.00 B)

  Total parameters: 305,793


ValueError: The filepath provided must end in `.keras` (Keras model format). Received: filepath=../results/models/lstm_best.h5